In [2]:
import numpy as np
import pandas as pd
from  sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
# import BatchNormalization
from keras.layers import BatchNormalization
from keras.layers import Dropout


2024-03-16 20:45:54.532568: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-16 20:45:54.532619: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-16 20:45:54.534189: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-16 20:45:54.542958: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-16 20:45:55.356199: W tensorflow/compiler/tf2

In [3]:
# Load data
file_path = 'dataset.xlsx'
data = pd.read_excel(file_path)
data = data.astype({'x1': 'float', 'x2': 'float', 'x3': 'float', 'y1': 'category', 'y2': 'float'})


In [4]:
# split
X = data[['x1', 'x2', 'x3']].values
Y_class = data['y1'].values
Y_reg = data['y2'].values
# print one line of data
i= 180
print(X[i], Y_class[i], Y_reg[i])

[-0.0700291  -0.1018302   0.01919956] 2 5.05797


In [5]:
# onehot encode the class labels
encoder = OneHotEncoder(sparse_output=False)
Y_class = encoder.fit_transform(Y_class.reshape(-1, 1))
# Y_class = Y_class.toarray()


In [6]:
print(Y_class[180])
print(Y_class[280])
print(Y_class[0])
print(Y_class[1])

[0. 0. 1. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0.]


In [7]:
X_train, X_test, Y_class_train, Y_class_test, Y_reg_train, Y_reg_test = train_test_split(X, Y_class, Y_reg, test_size=0.2, random_state=42)

In [8]:
# 线性回归模型
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
import numpy as np


In [9]:
print(X_train.shape, X_test.shape, Y_class_train.shape, Y_class_test.shape, Y_reg_train.shape, Y_reg_test.shape)

(448, 3) (112, 3) (448, 8) (112, 8) (448,) (112,)


In [11]:
linear_regression1 = LinearRegression()
linear_regression1.fit(X_train, Y_class_train)


LinearRegression()

ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets

In [8]:
# from nn_trainer import trainer_model

In [9]:
# nn model
# def create_model(input_dim, output_dim):
#     initializer = keras.initializers.he_normal(seed=42)
#     return trainer_model(
#         layers=[
#             Input(shape=(input_dim,)),
#         ]
#     )


In [24]:
input_layer = Input(shape=(3,))  

# hidden_layer_1 = Dense(1024, activation='relu')(input_layer)
# hidden_layer_1 = Dropout(0.3)(hidden_layer_1)
# hidden_layer_2 = Dense(512, activation='relu')(hidden_layer_1)
# hidden_layer_2 = BatchNormalization()(hidden_layer_2)
hidden_layer_3 = Dense(2048, activation='elu')(input_layer)
hidden_layer_3 = Dropout(0.3)(hidden_layer_3)


class_output = Dense(8, activation='softmax', name='class_output')(hidden_layer_3)  
reg_output = Dense(1, name='reg_output')(hidden_layer_3)  


model = Model(inputs=input_layer, outputs=[class_output, reg_output])



In [25]:
model.compile(optimizer=Adam(learning_rate=0.0005),
              loss={'class_output': 'categorical_crossentropy', 'reg_output': 'mean_squared_error'},
              metrics={'class_output': 'accuracy', 'reg_output': 'mean_squared_error'})


In [26]:
history = model.fit(X_train, [Y_class_train, Y_reg_train],
                    validation_split=0.3,
                    epochs=100,
                    batch_size=32,
                    verbose=2)

Epoch 1/100


10/10 - 1s - 85ms/step - class_output_accuracy: 0.1438 - loss: 24.3972 - reg_output_mean_squared_error: 22.3205 - val_class_output_accuracy: 0.1185 - val_loss: 26.0671 - val_reg_output_mean_squared_error: 24.0008
Epoch 2/100
10/10 - 0s - 5ms/step - class_output_accuracy: 0.1374 - loss: 21.5143 - reg_output_mean_squared_error: 19.4516 - val_class_output_accuracy: 0.1259 - val_loss: 22.8857 - val_reg_output_mean_squared_error: 20.8261
Epoch 3/100
10/10 - 0s - 4ms/step - class_output_accuracy: 0.1470 - loss: 18.4436 - reg_output_mean_squared_error: 16.3927 - val_class_output_accuracy: 0.1259 - val_loss: 19.0748 - val_reg_output_mean_squared_error: 17.0208
Epoch 4/100
10/10 - 0s - 6ms/step - class_output_accuracy: 0.1246 - loss: 14.8039 - reg_output_mean_squared_error: 12.7621 - val_class_output_accuracy: 0.1333 - val_loss: 15.0008 - val_reg_output_mean_squared_error: 12.9468
Epoch 5/100
10/10 - 0s - 19ms/step - class_output_accuracy: 0.1310 - loss: 11.2696 - reg_output_mean_squared_error:

In [13]:
model.evaluate(X_test, [Y_class_test, Y_reg_test])

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - class_output_accuracy: 0.4621 - loss: 2.7716 - reg_output_mean_squared_error: 1.5776 


[3.121758460998535, 0.4285714328289032, 1.8190447092056274]

In [14]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 3)         │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense (Dense)       │ (None, 1024)      │   4,096 │ input_layer[0][0]    │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dropout (Dropout)   │ (None, 1024)      │       0 │ dense[0][0]          │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_1 (Dense)     │ (None, 512)       │ 524,800 │ dropout[0][0]        │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ batch_normalization │ (None, 512)       │   2,048 │ dense_1[0][0]        │
│ (BatchNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_2 (Dense)     │ (None, 286)       │ 146,718 │ batch_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dropout_1 (Dropout) │ (None, 286)       │       0 │ dense_2[0][0]        │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ class_output        │ (None, 8)         │   2,296 │ dropout_1[0][0]      │
│ (Dense)             │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reg_output (Dense)  │ (None, 1)         │     287 │ dropout_1[0][0]      │
└─────────────────────┴───────────────────┴─────────┴──────────────────────┘

 Total params: 2,038,689 (7.78 MB)

 Trainable params: 679,221 (2.59 MB)

 Non-trainable params: 1,024 (4.00 KB)

 Optimizer params: 1,358,444 (5.18 MB)

In [15]:
# pridection
i= 0
print(X[i], Y_class[i], Y_reg[i])


[ 0.00902681 -0.04783698 -0.01826162] [1. 0. 0. 0. 0. 0. 0. 0.] 1.0


In [16]:

x_new = X[1]

x_new = x_new.reshape(1, -1)

x_new

array([[ 0.02150743,  0.00640325, -0.04351044]])

In [17]:
y_class_p, y_reg_p = model.predict(x_new)

print(y_class_p[0])
print(y_reg_p[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
[0.10316101 0.05606828 0.02612244 0.13573425 0.03552317 0.21072368
 0.08906259 0.34360453]
[4.129134]


In [1]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
import numpy as np

# 生成一个简单的二分类数据集
X, y = make_classification(n_samples=100, n_features=2, n_informative=2, n_redundant=0, n_clusters_per_class=1)

# 将数据集分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 使用线性回归模型
linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)

# 预测测试集的值
y_pred = linear_regression.predict(X_test)

# 将回归输出转换为二分类输出，这里使用0.5作为分类阈值
y_pred_class = np.where(y_pred > 0.5, 1, 0)

# 计算准确率
accuracy = accuracy_score(y_test, y_pred_class)
print(f'Accuracy: {accuracy}')


Accuracy: 0.95


<!-- z -->